In [1]:
import numpy as np
from pyscf import gto, scf, fci
from tabulate import tabulate
from RDMFS import HF_U2RDM, MU_U2RDM, GU_U2RDM, BBC1_U2RDM
from PNOFs import PNOF2, PNOF4
from utils import spectral_clean, DM2_DiagSum, BST_FIDX, twoDM_Eigvals, twoDM_Eigvals_phys, DM2_DiagSum_phys
import scipy 

In [2]:
def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

In [3]:
# Get Hydrogen in mo basis from PySCF

mol = gto.Mole()
mol.unit = 'B' 
L = 3.0
mol.atom = f"""
# He 0 0 0 
H   {-L}   {-L}   0.0
H    {L}   {-L}   0.0
# H    {L}    {L}   0.0
# H   {-L}    {L}   0.0
# H   {-L}   {-L}   {L}
# H    {L}   {-L}   {L}
# H    {L}    {L}   {L}
# H   {-L}    {L}   {L}
"""
# this basis has 2 functions for Helium
# mol.basis = "ccpvdz"
mol.basis = "sto-3g"

# mol.basis = "631g"

mol.spin =  0
mol.verbose= 0
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
eri_ao = mol.intor('int2e')
S = mol.intor('int1e_ovlp')

## Run Hartree-Fock.
mf = scf.UHF(mol)
mf.kernel()


# Harvest HF quantities

h1 = mf.get_hcore()
M = h1.shape[0]
C_a = mf.mo_coeff[0,:,:]
C_b = mf.mo_coeff[1,:,:]

h1_a = C_a.T@h1@C_a
h1_b = C_b.T@h1@C_b
h1_mo = (h1_a, h1_b)


h2_mo = BST_FIDX((eri_ao, eri_ao, eri_ao),S@C_a,S@C_b)

In [4]:
dm1_ao = mf.make_rdm1()
dm2_ao = mf.make_rdm2()

dm1_mo = dm1_ao.copy()
# # dm1_mo=C[:,0:N]@dm1_ao@C[:,0:N].T
dm1_mo[0]=(C_a.T@(S.T@dm1_mo[0]@S))@C_a
dm1_mo[1]=(C_a.T@(S.T@dm1_mo[1]@S))@C_a

dm2_mo = BST_FIDX(dm2_ao,S@C_a,S@C_b)

# P = C@C.T
# print(P)

occ_mo_a, occ_mo_b, C_mo_a, C_mo_b = spectral_clean(dm1_mo)
print(occ_mo_a, np.sum(occ_mo_a))
print(occ_mo_b, np.sum(occ_mo_b))


[1. 0.] 1.0000000000000002
[1. 0.] 1.0000000000000002


In [5]:
dm2_HF_mo = HF_U2RDM(occ_mo_a, occ_mo_b,M)
dm2_MU_mo = MU_U2RDM(occ_mo_a, occ_mo_b,M)
dm2_GU_mo = GU_U2RDM(occ_mo_a, occ_mo_b,M)
dm2_BBC1_mo = BBC1_U2RDM(occ_mo_a, occ_mo_b,M,mol.nelec[0],mol.nelec[1])
# dm2_PNOF2_mo = PNOF2(occ_mo_a, occ_mo_b,M,mol.nelec[0],mol.nelec[1])


In [6]:
print(DM2_DiagSum(dm2_MU_mo), scipy.special.binom(np.sum(mol.nelec),2))
print(DM2_DiagSum(dm2_HF_mo), scipy.special.binom(np.sum(mol.nelec),2))
print(DM2_DiagSum(dm2_GU_mo), scipy.special.binom(np.sum(mol.nelec),2))
print(DM2_DiagSum(dm2_BBC1_mo), scipy.special.binom(np.sum(mol.nelec),2))
# print(DM2_DiagSum_phys(dm2_PNOF2_mo), scipy.special.binom(np.sum(mol.nelec),2))
print(DM2_DiagSum(dm2_mo), scipy.special.binom(np.sum(mol.nelec),2))

2.0000000000000013 1.0
2.000000000000001 1.0
2.000000000000001 1.0
2.0000000000000013 1.0
2.0000000000000004 1.0


In [7]:
# Run FCI  
cisolver = fci.direct_uhf.FCI()
cisolver.max_cycle = 100
cisolver.conv_tol = 1e-8
e_mo, ci_mo = cisolver.kernel(h1_mo, h2_mo, h1.shape[0], mol.nelec, ecore=mf.energy_nuc())
dm1_fci, dm2_fci  = cisolver.make_rdm12s(ci_mo, h1.shape[0], mol.nelec)
occ_mo_a, occ_mo_b, C_mo_a, C_mo_b = spectral_clean(dm1_fci)
print(occ_mo_a, np.sum(occ_mo_a))
print(occ_mo_b, np.sum(occ_mo_b))

[0.5 0.5] 1.0
[0.5 0.5] 1.0


In [8]:
dm2_HF_fci = HF_U2RDM(occ_mo_a, occ_mo_b, mol.nao)
dm2_MU_fci = MU_U2RDM(occ_mo_a, occ_mo_b, mol.nao)
dm2_GU_fci = GU_U2RDM(occ_mo_a, occ_mo_b, mol.nao)
dm2_BBC1_fci = BBC1_U2RDM(occ_mo_a, occ_mo_b, mol.nao, mol.nelec[0], mol.nelec[1])
dm2_PNOF2_mo = PNOF2(occ_mo_a, occ_mo_b,M,mol.nelec[0],mol.nelec[1])
dm2_PNOF4_mo = PNOF4(occ_mo_a, occ_mo_b,M,mol.nelec[0],mol.nelec[1])


0.499999999997368 1.000000000010528
0.4999999999973681 1.0000000000105276
0.499999999997368 1.000000000010528
0.4999999999973681 1.0000000000105276


In [9]:
print(np.sum(twoDM_Eigvals(dm2_HF_fci)), scipy.special.binom(np.sum(mol.nelec),2))
print(np.sum(twoDM_Eigvals(dm2_MU_fci)), scipy.special.binom(np.sum(mol.nelec),2))
print(np.sum(twoDM_Eigvals(dm2_GU_fci)), scipy.special.binom(np.sum(mol.nelec),2))
print(np.sum(twoDM_Eigvals(dm2_BBC1_fci)), scipy.special.binom(np.sum(mol.nelec),2))
print(np.sum(twoDM_Eigvals_phys(dm2_PNOF2_mo)), scipy.special.binom(np.sum(mol.nelec),2))
print(DM2_DiagSum_phys(dm2_PNOF2_mo))
print(DM2_DiagSum_phys(dm2_PNOF4_mo))

print(np.sum(twoDM_Eigvals(dm2_fci)), scipy.special.binom(np.sum(mol.nelec),2))

3.000000000000001 1.0
2.000000000000001 1.0
3.000000000000001 1.0
2.000000000000001 1.0
2.000000000000001 1.0
2.000000000000001
2.000000000000001
1.9999999999999996 1.0


In [10]:
print(len(dm2_PNOF2_mo))

3


In [11]:
print(DM2_DiagSum(dm2_MU_fci), scipy.special.binom(np.sum(mol.nelec),2))
print(DM2_DiagSum(dm2_HF_fci), scipy.special.binom(np.sum(mol.nelec),2))
print(DM2_DiagSum(dm2_GU_fci), scipy.special.binom(np.sum(mol.nelec),2))
print(DM2_DiagSum(dm2_BBC1_fci), scipy.special.binom(np.sum(mol.nelec),2))
print(DM2_DiagSum_phys(dm2_PNOF2_mo), scipy.special.binom(np.sum(mol.nelec),2))
print(DM2_DiagSum(dm2_fci), scipy.special.binom(np.sum(mol.nelec),2))

2.000000000000001 1.0
3.000000000000001 1.0
3.000000000000001 1.0
2.000000000000001 1.0
2.000000000000001 1.0
2.0 1.0


In [12]:
for i in range(0,M):
    for j in range(0,M):
        for k in range(0,M):
            for l in range(0,M):
                print(i,j,k,l,dm2_PNOF2_mo[1][i,j,k,l], dm2_PNOF4_mo[1][i,j,k,l])

0 0 0 0 0.500000000002632 0.500000000002632
0 0 0 1 0.0 0.0
0 0 1 0 0.0 0.0
0 0 1 1 2.63206123563009e-12 -0.3535533905923432
0 1 0 0 0.0 0.0
0 1 0 1 1.1102230246251565e-16 1.1102230246251565e-16
0 1 1 0 0.0 0.0
0 1 1 1 0.0 0.0
1 0 0 0 0.0 0.0
1 0 0 1 0.0 0.0
1 0 1 0 1.1102230246251565e-16 1.1102230246251565e-16
1 0 1 1 0.0 0.0
1 1 0 0 2.63206123563009e-12 -0.3535533905923432
1 1 0 1 0.0 0.0
1 1 1 0 0.0 0.0
1 1 1 1 0.4999999999973681 0.4999999999973681
